# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from my_example import *

from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

from pymoo.optimize import minimize


# função objetivo

In [15]:
# Definição da OF unitária

# Leitura dos dados
ARQ = "teste_wand.xlsx"
SHEET = 0
df = pd.read_excel(ARQ, sheet_name=SHEET)
df = tensao_adm_solo(df)
df.columns = [str(c).strip() for c in df.columns] # remover espaços nos nomes das colunas

# Parâmetros gerais
n_comb = 3
ro = 0.01
cob = 0.025
fck = 25000.0
fcd = fck / 1.4

#variáveis para teste
x = np.array([1, 1, 0.40])  # h_x, h_y, h_z
h_x, h_y, h_z = x

def obj_func_unitaria(x, row):

    # Lista para armazenar as tensões de cada combinação
    t_max_aux = []
    t_min_aux = []
    # loop para calcular a tensão de cada combinação e armazenar na lista
    for k in range(1, n_comb + 1):
        aux = f'c{k}'
        t_max_val, t_min_val = calcular_sigma_max(
                                                    float(row[f'Fz-{aux}']),
                                                    float(row[f'Mx-{aux}']),
                                                    float(row[f'My-{aux}']),
                                                    h_x, h_y
        )
        t_max_aux.append(t_max_val)
        t_min_aux.append(t_min_val)
    
    # Tensão máxima e mínima
    max_tensao = float(max(t_max_aux))
    min_tensao = float(min(t_min_aux))
    # identificador da combinação mais crítica
    idx_max = int(np.argmax(t_max_aux)) 
    
    # Esforços da combinação crítica de máximo (para punção)
    comb_max = f'c{idx_max+1}' # +1 para compensar o ínicio da contagem de "0" para "1"
    Fz_max = float(row[f'Fz-{comb_max}'])
    Mx_max = float(row[f'Mx-{comb_max}'])
    My_max = float(row[f'My-{comb_max}'])

    #chamar sigma_adm do df
    sigma_adm = float(row['sigma_adm (kPa)']) 

    #chamar geometria do pilar do df
    a_p = float(row['ap (m)'])
    b_p = float(row['bp (m)'])

    # verificação de tensão
    g_0 = abs(min_tensao / sigma_adm) if (min_tensao < 0) else (1.15 * (min_tensao / sigma_adm) - 1.0)
    g_1 = abs(max_tensao / sigma_adm) if (max_tensao < 0) else (1.15 * (max_tensao / sigma_adm) - 1.0)

    # restrição geométrica balanço sapata
    cap_value, cbp_value, g0b, g1b, g2b, g3b = restricao_geometrica_balanco_pilar_sapata(h_x, h_y, h_z, a_p, b_p)
    
    
    # restrição geométrica pilar-sapata
    g0ps, g1ps = restricao_geometrica_pilar_sapata(h_x, h_y, a_p, b_p)
    
    ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2,gp0, gp1, gp2, gp3, gp4, gp5, gp6 = restricao_puncao(   h_x=h_x, h_y=h_y, h_z=h_z,
                                                                                                                    a_p=a_p, b_p=b_p,
                                                                                                                    f_z=Fz_max, m_x=Mx_max, m_y=My_max,
                                                                                                                    ro=ro, cob=cob, fck=fck, fcd=fcd)

    # objetivo 
    vol = h_x * h_y * h_z

    return vol, idx_max+1, max_tensao, min_tensao, Fz_max, Mx_max, My_max, sigma_adm, a_p, b_p, cap_value, cbp_value, g_0, g_1, g0b, g1b, g2b, g3b, g0ps, g1ps, ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2, gp0, gp1, gp2, gp3, gp4, gp5, gp6


In [30]:
#Verificando saída da def OF unitária

vol, idx_max, max_tensao, min_tensao, Fz_max, Mx_max, My_max, sigma_adm, a_p, b_p, cap_value, cbp_value, g_0, g_1, g0b, g1b, g2b, g3b, g0ps, g1ps, ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2, gp0, gp1, gp2, gp3, gp4, gp5, gp6 = obj_func_unitaria(x, df.iloc[2])


print(f" vol: {vol}\n combinação desfavorável: C{idx_max}\n tensão máx: {max_tensao}\n tensão min:{min_tensao}\n", 
      f"f_z máx: {Fz_max}\n m_x máx: {Mx_max}\n m_y máx: {My_max}\n",
      f"sigma_adm: {sigma_adm}\n a_p: {a_p}\n b_p: {b_p}\n cap_value: {cap_value}\n cbp_value {cbp_value}\n g_0: {g_0}\n g_1: {g_1}\n",
      f"g0b: {g0b}\n g1b: {g1b}\n g2b: {g2b}\n g3b: {g3b}\n g0ps: {g0ps}\n g1ps: {g1ps}\n",
      f"ke: {ke}\n kx: {kx}\n ky: {ky}\n wpx: {wpx}\n wpy: {wpy}\n u: {u}\n",
      f"talsd1: {talsd1}\n talrd1: {talrd1}\n talsd2: {talsd2}\n talrd2: {talrd2}\n",
      f"gp0: {gp0}\n gp1: {gp1}\n gp2: {gp2}\n gp3: {gp3}\n gp4: {gp4}\n gp5: {gp5}\n gp6: {gp6}\n"
      )


 vol: 0.4
 combinação desfavorável: C3
 tensão máx: 2990.7000000000003
 tensão min:407.0000000000003
 f_z máx: 1698.9
 m_x máx: -214.6
 m_y máx: 0.7
 sigma_adm: 600.0
 a_p: 0.35000000000000003
 b_p: 2.32
 cap_value: 0.32499999999999996
 cbp_value -0.6599999999999999
 g_0: -0.2199166666666662
 g_1: 4.732175000000001
 g0b: -0.59375
 g1b: -1.8249999999999997
 g2b: -0.38461538461538447
 g3b: -1.3030303030303032
 g0ps: -0.6499999999999999
 g1ps: 1.3199999999999998
 ke: 1.7302967433402214
 kx: 0.8
 ky: 0.45
 wpx: 11.74457121724624
 wpy: 7.42791807156732
 u: 7.696194490192345
 talsd1: 0.5497868023304636
 talrd1: 6.57724388086839
 talsd2: 1.0821019108280256
 talrd2: 4.339285714285714
 gp0: -0.9390625
 gp1: -1.12375
 gp2: -0.5
 gp3: -0.13485162832988928
 gp4: -1.0
 gp5: -0.9164107622754175
 gp6: -0.7506267201383974



In [ ]:
# Loop para rodar a OF unitária no otrimizador,


#pymoo mono objetivo usando dataframe

# Leitura dos dados
ARQ = "teste_wand.xlsx"
SHEET = 0
df = pd.read_excel(ARQ, sheet_name=SHEET)
df = tensao_adm_solo(df)
df.columns = [str(c).strip() for c in df.columns] # remover espaços nos nomes das colunas

# Parâmetros gerais
n_comb = 3
ro = 0.01
cob = 0.025
fck = 25000.0
fcd = fck / 1.4
row = len(df)

def extrair_params(row):
    """
    Extrai parâmetros de uma linha do DataFrame.
    Retorna os valores das colunas correspondentes (ou valores padrão, se faltarem).
    """
    # Nome da coluna e valor padrão
    colunas = {
        "h_z": ("h_z", 0.40),
        "a_p": ("ap (m)", 0.25),
        "b_p": ("bp (m)", 1.25),
        "Fz":  ("Fz-c1", 500.0),
        "Mx":  ("Mx-c1", 0.0),
        "My":  ("My-c1", 0.0),
        "ro":  ("ro", 0.010),
        "cob": ("cob", 0.030),
        "fck": ("fck", 25000.0),
    }

    p = {}
    for chave, (coluna, padrao) in colunas.items():
        # verifica se a coluna existe e o valor não é nulo
        if coluna in row and pd.notna(row[coluna]):
            p[chave] = float(row[coluna])
        else:
            p[chave] = padrao

    # Calcula fcd a partir do fck
    p["fcd"] = p["fck"] / 1.4

    return p


# PROBLEMA (1 objetivo)
class MyProblem(ElementwiseProblem):
    def __init__(self, row_series, params, xl=np.array([0.6, 0.6, 0.6]), xu=np.array([3.0, 3.0, 3.0])):
        super().__init__(n_var=3, n_obj=1, n_constr= 15, xl=xl, xu=xu)
        self.row = row_series

    def _evaluate(self, x, out, *args, **kwargs):
        
        h_x, h_y, h_z  = float(x[0]), float(x[1]), float(x[2])

        # Executando OF    
        vol, idx_max, max_tensao, min_tensao, Fz_max, Mx_max, My_max, sigma_adm, a_p, b_p, cap_value, cbp_value, g_0, g_1, g0b, g1b, g2b, g3b, g0ps, g1ps, ke, kx, ky, wpx, wpy, u, talsd1, talrd1, talsd2, talrd2, gp0, gp1, gp2, gp3, gp4, gp5, gp6 = obj_func_unitaria(x, row=self.row)
        

        F = float(vol) if np.isfinite(vol) else 1e50
        G = np.asarray([g_0, g_1, g0b, g1b, g2b, g3b, g0ps, g1ps, gp0, gp1, gp2, gp3, gp4, gp5, gp6  ], dtype=float)
        G[~np.isfinite(G)] = 1e50

        out["F"] = F
        out["G"] = G

def chamando_GA():
    return GA(
        pop_size=40,
        n_offsprings=10,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
        eliminate_duplicates=True
    )

# LOOP NAS LINHAS DO DF

resultados = []

for i in df.iterrows():
    row_dict = df.iloc[i].to_dict()
    params = extrair_params(row_dict)

    problem = MyProblem(row_series=df.iloc[i])
    algorithm = chamando_GA()

    res = minimize(problem, algorithm, ('n_gen', 40), seed=1, verbose=False)

    # Cada linha tem seu próprio X (h_x, h_y) e F, G
    X = res.X
    F = res.F
    G = res.G

    # garantir formato consistente
    if np.ndim(X) > 1:
        x_best = X[0]
    else:
        x_best = X

    if np.ndim(F) > 1:
        f_best = F[0]
    else:
        f_best = F

    if np.ndim(G) > 1:
        g_best = G[0]
    else:
        g_best = G

    # garantir que g_best seja 1-D para indexação segura
    g_best = np.ravel(g_best)

    resultados.append({
        "linha": i,
        "h_x": float(x_best[0]),
        "h_y": float(x_best[1]),
        "f1_volume": float(f_best[0]),
        "g0": g_best[0],
        "g1": g_best[1],
        "g2": g_best[2],
        "g3": g_best[3],
        "g4": g_best[4],
        "g5": g_best[5],
        "g6": g_best[6],
        "g7": g_best[7],
        "g8": g_best[8],
        "g9": g_best[9],
        "g10": g_best[10],
        "g11": g_best[11],
        "g12": g_best[12],
        **params
    })


# print("Best solution found: \nX = %s\nF = %s \nG = %s" % (res.X, res.F, res.G))
resultados_df = pd.DataFrame(resultados)
print(resultados_df)


IndexError: .iloc requires numeric indexers, got ['P01' 0.25 1.2 10 'argila' 2.9 25.255 485.9 -0.3 4.1 511.6 -32.4 -0.4
 511.6 -32.4 -0.4 200.0]

In [ ]:
# --- Loop robusto para rodar of unitária em todsas TODAS as linhas ---
resultados = []
falhas = []

colunas_resultado = [
    'comb_crit', 't_max', 't_min', 'Fz_max', 'Mx_max', 'My_max',
    'sigma_adm', 'a_p', 'b_p', 'cap_value', 'cbp_value',
    'g_0', 'g_1', 'g0b', 'g1b', 'g2b', 'g3b', 'g0ps', 'g1ps',
    'ke', 'kx', 'ky', 'wpx', 'wpy', 'u',
    'talsd1', 'talrd1', 'talsd2', 'talrd2',
    'gp0', 'gp1', 'gp2', 'gp3', 'gp4', 'gp5', 'gp6'
]  # 36 colunas

total = len(df)
for i, row in df.iterrows():
    try:
        out = obj_func_unitaria(x, row)
        if len(out) != len(colunas_resultado):
            raise ValueError(f"Retorno com {len(out)} itens, esperado {len(colunas_resultado)}")
        resultados.append(out)
    except Exception as e:
        falhas.append((i, str(e)))
        # Preenche com NaNs para manter o alinhamento
        resultados.append(tuple([np.nan]*len(colunas_resultado)))

# Monta o DataFrame dos resultados e concatena
resultados_df = pd.DataFrame(resultados, columns=colunas_resultado)
df_final = pd.concat([df.reset_index(drop=True), resultados_df], axis=1)

# Relatório rápido
print(f"Linhas processadas: {total}")
print(f"Sucesso: {total - len(falhas)} | Falhas: {len(falhas)}")
if falhas:
    print("Falhas (primeiras 10):")
    for j, (idx, msg) in enumerate(falhas[:10], 1):
        print(f"  {j:02d}) idx {idx}: {msg}")

# Opcional: salvar
df_final.to_excel("resultados_unitarios.xlsx", index=False)
